In [1]:
# !pip install easygui --proxy http://10.2.176.162:8080

In [2]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
import easygui

In [4]:
#parse the twb file
file = easygui.fileopenbox(filetypes=['YOURWORKBOOKNAME.twb'])
tree = ET.parse(file)
root = tree.getroot()

ParseError: syntax error: line 1, column 0 (<string>)

In [ ]:
#create a dictionary of name and tableau generated name
calcDict = {}
for item in root.findall('.//column[@caption]'):
    if item.find(".//calculation") is None:
        continue
    else:
        calcDict[item.attrib['name']] = '[' + item.attrib['caption'] + ']'
calcDict

In [ ]:
#list of calc's name, tableau generated name, and calculation/formula
calcList = []
for item in root.findall('.//column[@caption]'):
    if item.find(".//calculation") is None:
        continue
    else:
        if item.find(".//calculation[@formula]") is None:
            continue
        else:
            calc_caption = '[' + item.attrib['caption'] + ']'
            calc_name = item.attrib['name']
            calc_raw_formula = item.find(".//calculation").attrib['formula']
            calc_comment = ''
            calc_formula = ''
            for line in calc_raw_formula.split('\r\n'):
                if line.startswith('//'):
                    calc_comment = calc_comment + line + ' '
                else:
                    calc_formula = calc_formula + line + ' '    
        for name, caption in calcDict.items():
            calc_formula = calc_formula.replace(name, caption)
            calc_row = (calc_caption, calc_name, calc_formula, calc_comment)
            calcList.append(list(calc_row))

#convert the list of calcs into a data frame
data = calcList
data = pd.DataFrame(data, columns=['Name', 'Remote Name', 'Formula', 'Comment'])

#remove duplicate rows from data frame
data = data.drop_duplicates(subset=None, keep='first', inplace=False)

# export to csv
base = os.path.basename(file)
os.path.splitext(base)
filename = os.path.splitext(base)[0]
data.to_csv(filename + '.csv', encoding="utf-8")